In [47]:
# ████ ███ ███ ███ ███ ┼┼ █┼┼ ███ ███ ███ █┼┼█ ███ █┼┼█ ████
# █┼▄▄ █▄┼ █▄┼ █▄█ ┼█┼ ┼┼ █┼┼ █▄┼ █▄█ █▄┼ ██▄█ ┼█┼ ██▄█ █┼▄▄
# █▄▄█ █┼█ █▄▄ █┼█ ┼█┼ ┼┼ █▄█ █▄▄ █┼█ █┼█ █┼██ ▄█▄ █┼██ █▄▄█

In [48]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import scipy 
import sklearn
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import confusion_matrix, accuracy_score,precision_score, classification_report,log_loss,roc_curve, roc_auc_score,cohen_kappa_score, f1_score,recall_score
from sklearn.model_selection import train_test_split, GridSearchCV
import statsmodels.api as sm 
from sklearn.datasets import make_classification
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree

In [49]:
plt.rcParams['figure.figsize'] = [15,8]

In [50]:
df = pd.read_csv('Admission_Predict.csv')
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,1
1,2,324,107,4,4.0,4.5,8.87,1,1
2,3,316,104,3,3.0,3.5,8.00,1,0
3,4,322,110,3,3.5,2.5,8.67,1,1
4,5,314,103,2,2.0,3.0,8.21,0,0


In [51]:
df.drop("Serial No.", axis = 1, inplace = True)

In [52]:
X = df.drop('Chance of Admit',axis = 1)
y = df['Chance of Admit']


In [53]:
X_train,X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=1)

In [54]:
perf_score = pd.DataFrame(columns=["Model", "Accuracy","Recall","Precision","F1 Score"] )

In [55]:
def per_measures(model,test,pred):
    accuracy    =accuracy_score(test,pred)
    f1score     =f1_score(test,pred)
    recall      =recall_score(test,pred)
    precision   =precision_score(test,pred)
    # aic         =model.aic
    # reliability = cohen_kappa_score(test,pred)
    return (accuracy,recall,precision,f1score,)

In [56]:
def update_performance (name,
                        model,
                        test,
                        pred
                           ):

    # assign 'comp_perf' as global variable
    global perf_score


    # append the results to the dataframe 'score_card'
    # 'ignore_index = True' do not consider the index labels
    perf_score = perf_score.append({'Model'       : name,
                                    'Accuracy'    : per_measures(model,test,pred)[0],
                                    'Recall'      : per_measures(model,test,pred)[1],
                                    'Precision'   : per_measures(model,test,pred)[2],
                                    'F1 Score'    : per_measures(model,test,pred)[3]
                                    # 'AIC'         : per_measures(model,ytest,ypred)[4],
                                    # 'Reliability' : per_measures(model,ytest,ypred)[5]

                                     },
                                   ignore_index = True)

In [57]:
rf = RandomForestClassifier(random_state=10)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=10)

In [58]:
ypred_rf_train = rf.predict(X_train)

print(accuracy_score(y_train, ypred_rf_train))
print("********************************************")
print(confusion_matrix(y_train, ypred_rf_train))
print("********************************************")
print(classification_report(y_train, ypred_rf_train))

1.0
********************************************
[[154   0]
 [  0 126]]
********************************************
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       154
           1       1.00      1.00      1.00       126

    accuracy                           1.00       280
   macro avg       1.00      1.00      1.00       280
weighted avg       1.00      1.00      1.00       280



In [59]:
ypred_rf_train = rf.predict(X_train)

print(accuracy_score(y_train, ypred_rf_train))
print("********************************************")
print(confusion_matrix(y_train, ypred_rf_train))
print("********************************************")
print(classification_report(y_train, ypred_rf_train))

1.0
********************************************
[[154   0]
 [  0 126]]
********************************************
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       154
           1       1.00      1.00      1.00       126

    accuracy                           1.00       280
   macro avg       1.00      1.00      1.00       280
weighted avg       1.00      1.00      1.00       280



In [60]:
ypred_rf_test = rf.predict(X_test)

print(accuracy_score(y_test, ypred_rf_test))
print("********************************************")
print(confusion_matrix(y_test, ypred_rf_test))
print("********************************************")
print(classification_report(y_test, ypred_rf_test))

0.8666666666666667
********************************************
[[60  6]
 [10 44]]
********************************************
              precision    recall  f1-score   support

           0       0.86      0.91      0.88        66
           1       0.88      0.81      0.85        54

    accuracy                           0.87       120
   macro avg       0.87      0.86      0.86       120
weighted avg       0.87      0.87      0.87       120



In [61]:
update_performance(name = 'Random Forest' ,model = rf, test = y_test, pred=ypred_rf_test)

In [62]:
perf_score

,Model,Accuracy,Recall,Precision,F1 Score
0,Random Forest,0.866667,0.814815,0.88,0.846154


In [63]:
rf = RandomForestClassifier(oob_score=True, random_state=10)

rf.fit(X_train, y_train)

ypred_rf = rf.predict(X_test)
print('accuracy_score  ',accuracy_score(y_test ,ypred_rf))
print('oob_score_  ',rf.oob_score_)

accuracy_score   0.8666666666666667
oob_score_   0.8464285714285714


In [64]:
params =             [{'criterion': ['entropy', 'gini'],
                     'n_estimators': [100],
                     'max_depth': [10, 15],
                     'max_features': ['sqrt', 'log2'],
                     'min_samples_split': [2, 8],
                     'min_samples_leaf': [5, 9],
                     'max_leaf_nodes': [8, 11]}]

In [65]:
rf = RandomForestClassifier(random_state=10)
rf_cv = GridSearchCV(rf, params, cv = 5, scoring='accuracy')
rf_cv.fit(X_train, y_train)
rf_cv.best_params_

{'criterion': 'gini',
 'max_depth': 10,
 'max_features': 'sqrt',
 'max_leaf_nodes': 8,
 'min_samples_leaf': 9,
 'min_samples_split': 2,
 'n_estimators': 100}

In [66]:
rf = RandomForestClassifier(random_state=10,criterion = 'gini',
 max_depth = 10,
 max_features = 'sqrt',
 max_leaf_nodes = 8,
 min_samples_leaf = 9,
 min_samples_split = 2,
 n_estimators = 100)

rf.fit(X_train, y_train)

ypred_rf = rf.predict(X_test)
print('accuracy_score  ',accuracy_score(y_test ,ypred_rf))

accuracy_score   0.875


In [67]:
print(classification_report(y_test, ypred_rf))

              precision    recall  f1-score   support

           0       0.87      0.91      0.89        66
           1       0.88      0.83      0.86        54

    accuracy                           0.88       120
   macro avg       0.88      0.87      0.87       120
weighted avg       0.88      0.88      0.87       120



In [68]:
update_performance(name = 'Random Forest - Tuned' ,model = rf, test = y_test, pred=ypred_rf)

In [69]:
perf_score

,Model,Accuracy,Recall,Precision,F1 Score
0,Random Forest,0.866667,0.814815,0.880000,0.846154
1,Random Forest - Tuned,0.875000,0.833333,0.882353,0.857143


## Bagging Classifier

In [70]:
from sklearn.ensemble import BaggingClassifier

In [71]:
dt = DecisionTreeClassifier(random_state=10)

bc = BaggingClassifier(dt)
bc.fit(X_train, y_train)

ypred_bc = bc.predict(X_test)
print('accuracy_score  ',accuracy_score(y_test ,ypred_bc))

accuracy_score   0.85


In [72]:
update_performance(name = 'Bagging Classifier-dt', model = bc, test=y_test,pred=ypred_bc)

In [73]:
perf_score

,Model,Accuracy,Recall,Precision,F1 Score
0,Random Forest,0.866667,0.814815,0.880000,0.846154
1,Random Forest - Tuned,0.875000,0.833333,0.882353,0.857143
2,Bagging Classifier-dt,0.850000,0.814815,0.846154,0.830189


In [74]:
from sklearn.ensemble import BaggingClassifier
knn=KNeighborsClassifier ()
bag_knn=BaggingClassifier(knn, random_state=10) # by default the estimator is Decision
bag_knn.fit(X_train,y_train)

ypred_bag_knn = bag_knn.predict(X_test)
print('accuracy_score  ',accuracy_score(y_test ,ypred_bag_knn))

accuracy_score   0.875


In [75]:
update_performance(name = 'Bagging Classifier-KNN ', model = bag_knn, test=y_test,pred=ypred_bag_knn)

In [76]:
perf_score

,Model,Accuracy,Recall,Precision,F1 Score
0,Random Forest,0.866667,0.814815,0.880000,0.846154
1,Random Forest - Tuned,0.875000,0.833333,0.882353,0.857143
2,Bagging Classifier-dt,0.850000,0.814815,0.846154,0.830189
3,Bagging Classifier-KNN,0.875000,0.814815,0.897959,0.854369


## Bagging LR Model .LR as base Model

In [77]:
logr = LogisticRegression()

bag_logr = BaggingClassifier(logr, random_state=10)
bag_logr.fit(X_train, y_train)

ypred_bag_logr = bag_logr.predict(X_test)

print(accuracy_score(y_test, ypred_bag_logr))

0.8666666666666667


In [78]:
update_performance(name = 'Bagging LR Model .LR as base Model', model = bag_logr, test=y_test,pred=ypred_bag_logr)

In [79]:
perf_score

,Model,Accuracy,Recall,Precision,F1 Score
0,Random Forest,0.866667,0.814815,0.880000,0.846154
1,Random Forest - Tuned,0.875000,0.833333,0.882353,0.857143
2,Bagging Classifier-dt,0.850000,0.814815,0.846154,0.830189
3,Bagging Classifier-KNN,0.875000,0.814815,0.897959,0.854369
4,Bagging LR Model .LR as base Model,0.866667,0.870370,0.839286,0.854545


In [80]:
0.5*np.log(2.5)

0.45814536593707755

In [81]:
# AOS(amount of say) = 0.5 * logbase e((1-total eror)/totaleror)

In [82]:
0.2575/0.515

0.5

# ADA Boost

In [83]:
from sklearn.ensemble import AdaBoostClassifier

In [84]:
abcl = AdaBoostClassifier(dt, random_state=10)
abcl.fit(X_train, y_train)

ypred_abcl = abcl.predict(X_test)
print(accuracy_score(y_test, ypred_abcl))

0.85


In [85]:
update_performance(name = 'Ada Boost Classifier ', model = abcl, test=y_test,pred=ypred_abcl)

In [86]:
perf_score

,Model,Accuracy,Recall,Precision,F1 Score
0,Random Forest,0.866667,0.814815,0.880000,0.846154
1,Random Forest - Tuned,0.875000,0.833333,0.882353,0.857143
2,Bagging Classifier-dt,0.850000,0.814815,0.846154,0.830189
3,Bagging Classifier-KNN,0.875000,0.814815,0.897959,0.854369
4,Bagging LR Model .LR as base Model,0.866667,0.870370,0.839286,0.854545
5,Ada Boost Classifier,0.850000,0.796296,0.860000,0.826923


## ADA Boost with RF

In [87]:
rf = RandomForestClassifier(random_state=10)
abcl_rf = AdaBoostClassifier(rf, random_state=10)
abcl_rf.fit(X_train, y_train)

ypred_abcl_rf = abcl_rf.predict(X_test)
print(accuracy_score(y_test, ypred_abcl_rf))

0.8666666666666667


In [88]:
update_performance(name = 'Ada Boost Classifier RF', model = abcl_rf, test=y_test,pred=ypred_abcl_rf)

In [90]:
perf_score

,Model,Accuracy,Recall,Precision,F1 Score
0,Random Forest,0.866667,0.814815,0.880000,0.846154
1,Random Forest - Tuned,0.875000,0.833333,0.882353,0.857143
2,Bagging Classifier-dt,0.850000,0.814815,0.846154,0.830189
3,Bagging Classifier-KNN,0.875000,0.814815,0.897959,0.854369
4,Bagging LR Model .LR as base Model,0.866667,0.870370,0.839286,0.854545
5,Ada Boost Classifier,0.850000,0.796296,0.860000,0.826923
6,Ada Boost Classifier RF,0.866667,0.814815,0.880000,0.846154


## Multinomial Naive Bayes

In [92]:
mnb = MultinomialNB()
abcl_mnb = AdaBoostClassifier(mnb, random_state=10)
abcl_mnb.fit(X_train, y_train)

ypred_abcl_mnb = abcl_mnb.predict(X_test)
print(accuracy_score(y_test, ypred_abcl_mnb))

update_performance(name = 'Ada Boost Classifier MNB', model = abcl_mnb, test=y_test,pred=ypred_abcl_mnb)

perf_score

0.45


,Model,Accuracy,Recall,Precision,F1 Score
0,Random Forest,0.866667,0.814815,0.880000,0.846154
1,Random Forest - Tuned,0.875000,0.833333,0.882353,0.857143
2,Bagging Classifier-dt,0.850000,0.814815,0.846154,0.830189
3,Bagging Classifier-KNN,0.875000,0.814815,0.897959,0.854369
4,Bagging LR Model .LR as base Model,0.866667,0.870370,0.839286,0.854545
5,Ada Boost Classifier,0.850000,0.796296,0.860000,0.826923
6,Ada Boost Classifier RF,0.866667,0.814815,0.880000,0.846154
7,Ada Boost Classifier MNB,0.450000,1.000000,0.450000,0.620690


## Gradient Boosting

In [96]:
from sklearn.ensemble import GradientBoostingClassifier
gbc1 = GradientBoostingClassifier (n_estimators=50, random_state=10)
gbc1.fit(X_train,y_train)
ypred_gbc1 = gbc1.predict(X_test)
print(accuracy_score(y_test,ypred_gbc1))

0.85


In [98]:
gbc1 = GradientBoostingClassifier(n_estimators=50,learning_rate=0.5, random_state=1,max_depth=3)
gbc1.fit(X_train,y_train)
ypred_gbc1 = gbc1.predict(X_test)
print(accuracy_score(y_test, ypred_gbc1))

0.825


In [100]:
update_performance (name = 'Gradient Boosting', model = gbc1, test=y_test, pred =ypred_gbc1)

In [101]:
perf_score

,Model,Accuracy,Recall,Precision,F1 Score
0,Random Forest,0.866667,0.814815,0.880000,0.846154
1,Random Forest - Tuned,0.875000,0.833333,0.882353,0.857143
2,Bagging Classifier-dt,0.850000,0.814815,0.846154,0.830189
3,Bagging Classifier-KNN,0.875000,0.814815,0.897959,0.854369
4,Bagging LR Model .LR as base Model,0.866667,0.870370,0.839286,0.854545
5,Ada Boost Classifier,0.850000,0.796296,0.860000,0.826923
6,Ada Boost Classifier RF,0.866667,0.814815,0.880000,0.846154
7,Ada Boost Classifier MNB,0.450000,1.000000,0.450000,0.620690
8,Gradient Boosting,0.825000,0.759259,0.836735,0.796117
